In [1]:
from __future__ import print_function

import tensorflow as tf

c = tf.constant('Hello, world!')

with tf.Session() as sess:

    print(sess.run(c))

b'Hello, world!'


 ## 概念概览

TensorFlow 的名称源自**张量**，张量是任意维度的数组。借助 TensorFlow，您可以操控具有大量维度的张量。即便如此，在大多数情况下，您会使用以下一个或多个低维张量：

  * **标量**是零维数组（零阶张量）。例如，`\'Howdy\'` 或 `5`
  * **矢量**是一维数组（一阶张量）。例如，`[2, 3, 5, 7, 11]` 或 `[5]`
  * **矩阵**是二维数组（二阶张量）。例如，`[[3.1, 8.2, 5.9][4.3, -2.7, 6.5]]`

TensorFlow **指令**会创建、销毁和操控张量。典型 TensorFlow 程序中的大多数代码行都是指令。

TensorFlow **图**（也称为**计算图**或**数据流图**）是一种图数据结构。很多 TensorFlow 程序由单个图构成，但是 TensorFlow 程序可以选择创建多个图。图的节点是指令；图的边是张量。张量流经图，在每个节点由一个指令操控。一个指令的输出张量通常会变成后续指令的输入张量。TensorFlow 会实现**延迟执行模型**，意味着系统仅会根据相关节点的需求在需要时计算节点。

张量可以作为**常量**或**变量**存储在图中。您可能已经猜到，常量存储的是值不会发生更改的张量，而变量存储的是值会发生更改的张量。不过，您可能没有猜到的是，常量和变量都只是图中的一种指令。常量是始终会返回同一张量值的指令。变量是会返回分配给它的任何张量的指令。

要定义常量，请使用 `tf.constant` 指令，并传入它的值。例如：

```
  x = tf.constant([5.2])
```

同样，您可以创建如下变量：

```
  y = tf.Variable([5])
```

或者，您也可以先创建变量，然后再如下所示地分配一个值（注意：您始终需要指定一个默认值）：

```
  y = tf.Variable([0])
  y = y.assign([5])
```

定义一些常量或变量后，您可以将它们与其他指令（如 `tf.add`）结合使用。在评估 `tf.add` 指令时，它会调用您的 `tf.constant` 或 `tf.Variable` 指令，以获取它们的值，然后返回一个包含这些值之和的新张量。

图必须在 TensorFlow **会话**中运行，会话存储了它所运行的图的状态：

```
将 tf.Session() 作为会话：
  initialization = tf.global_variables_initializer()
  print(y.eval())
```

在使用 `tf.Variable` 时，您必须在会话开始时调用 `tf.global_variables_initializer`，以明确初始化这些变量，如上所示。

**注意：**会话可以将图分发到多个机器上执行（假设程序在某个分布式计算框架上运行）。有关详情，请参阅[分布式 TensorFlow](https://www.tensorflow.org/deploy/distributed)。

### 总结

TensorFlow 编程本质上是一个两步流程：

1. 将常量、变量和指令整合到一个图中。
2. 在一个会话中评估这些常量、变量和指令。


In [5]:
#图、张量、 常量、变量、 指令
# Create a graph.
g = tf.Graph()

# Establish the graph as the "default" graph.
with g.as_default():
  # 以下有三个操作，创建两个常量，然后相加
  x = tf.constant(8, name="x_const")
  y = tf.constant(5, name="y_const")
  z = tf.constant(4, name="z_const")
    #现在又加了一个数，先加前两个，再与第三个相加
  sum_x_y = tf.add(x, y, name="x_y_sum")
  sum_x_y_z = tf.add(sum_x_y, z, name="x_y_z_sum")

  # Now create a session.
  # The session will run the default graph.
  with tf.Session() as sess:
    print(sum_x_y_z.eval())

17


In [9]:
#矢量加法
with tf.Graph().as_default():
  # Create a six-element vector (1-D tensor).
  primes = tf.constant([2, 3, 5, 7, 11, 13], dtype=tf.int32)

  # Create another six-element vector. Each element in the vector will be
  # initialized to 1. The first argument is the shape of the tensor (more
  # on shapes below).创建一个长度为6，每个元素都是1的矢量，注意one，指定了内容是1，如果是zero，则指定内容是0
  ones = tf.ones([6], dtype=tf.int32)

  # 所以矢量相加，就是它们的元素一一对应相加
  just_beyond_primes = tf.add(primes, ones)

  # Create a session to run the default graph.
  with tf.Session() as sess:
    print(just_beyond_primes.eval())

[ 3  4  6  8 12 14]


In [14]:
#显示张量的形状
with tf.Graph().as_default():
  # A scalar (0-D tensor).
  scalar = tf.zeros([])

  # A vector with 3 elements.
  vector = tf.zeros([3])

  # A matrix with 2 rows and 3 columns.
  matrix = tf.ones([2, 3])

  with tf.Session() as sess:
    print('scalar has shape', scalar.get_shape(), 'and value:\n', scalar.eval())
    print('vector has shape', vector.get_shape(), 'and value:\n', vector.eval())
    print('matrix has shape', matrix.get_shape(), 'and value:\n', matrix.eval())

scalar has shape () and value:
 0.0
vector has shape (3,) and value:
 [0. 0. 0.]
matrix has shape (2, 3) and value:
 [[1. 1. 1.]
 [1. 1. 1.]]


### 广播

在数学中，您只能对形状相同的张量执行元素级运算（例如，*相加*和*等于*）。不过，在 TensorFlow 中，您可以对张量执行传统意义上不可行的运算。TensorFlow 支持**广播**（一种借鉴自 Numpy 的概念）。利用广播，元素级运算中的较小数组会增大到与较大数组具有相同的形状。例如，通过广播：

* 如果指令需要大小为 `[6]` 的张量，则大小为 `[1]` 或 `[]` 的张量可以作为运算数。
* 如果指令需要大小为 `[4, 6]` 的张量，则以下任何大小的张量都可以作为运算数。
  * `[1, 6]`
  * `[6]`
  * `[]`
* 如果指令需要大小为 `[3, 5, 6]` 的张量，则以下任何大小的张量都可以作为运算数。

  * `[1, 5, 6]`
  * `[3, 1, 6]`
  * `[3, 5, 1]`
  * `[1, 1, 1]`
  * `[5, 6]`
  * `[1, 6]`
  * `[6]`
  * `[1]`
  * `[]`
  
**注意：**当张量被广播时，从概念上来说，系统会**复制**其条目（出于性能考虑，实际并不复制。广播专为实现性能优化而设计）。

有关完整的广播规则集，请参阅简单易懂的 [Numpy 广播文档](http://docs.scipy.org/doc/numpy-1.10.1/user/basics.broadcasting.html)。

In [16]:
#所以，广播就是把两个不同弄的矩阵进行运算，之前数据分析时学过，如果不匹配，好像会生成 nan

with tf.Graph().as_default():
  # Create a six-element vector (1-D tensor).
  primes = tf.constant([2, 3, 5, 7, 11, 13], dtype=tf.int32)

  # Create a constant scalar with value 1.注意这里只是一个单独的数 1，而上面是矢量
  ones = tf.constant(1, dtype=tf.int32)

  # Add the two tensors. The resulting tensor is a six-element vector.
  just_beyond_primes = tf.add(primes, ones)

  with tf.Session() as sess:
    print(just_beyond_primes.eval())
    print(ones.eval())

[ 3  4  6  8 12 14]
1


In [17]:
#进行矩阵乘法，与线性代数原则一致
with tf.Graph().as_default():
  # Create a matrix (2-d tensor) with 3 rows and 4 columns.
  x = tf.constant([[5, 2, 4, 3], [5, 1, 6, -2], [-1, 3, -1, -2]],
                  dtype=tf.int32)

  # Create a matrix with 4 rows and 2 columns.
  y = tf.constant([[2, 2], [3, 5], [4, 5], [1, 6]], dtype=tf.int32)

  # Multiply `x` by `y`. 
  # The resulting matrix will have 3 rows and 2 columns.
  matrix_multiply_result = tf.matmul(x, y)

  with tf.Session() as sess:
    print(matrix_multiply_result.eval())

[[35 58]
 [35 33]
 [ 1 -4]]


In [18]:
#可以使用 tf.reshape 方法改变张量的形状。 例如，您可以将 8x2 张量变形为 2x8 张量或 4x4 张量
with tf.Graph().as_default():
  # Create an 8x2 matrix (2-D tensor).
  matrix = tf.constant([[1,2], [3,4], [5,6], [7,8],
                        [9,10], [11,12], [13, 14], [15,16]], dtype=tf.int32)

  # Reshape the 8x2 matrix into a 2x8 matrix.
  reshaped_2x8_matrix = tf.reshape(matrix, [2,8])
  
  # Reshape the 8x2 matrix into a 4x4 matrix
  reshaped_4x4_matrix = tf.reshape(matrix, [4,4])

  with tf.Session() as sess:
    print("Original matrix (8x2):")
    print(matrix.eval())
    print("Reshaped matrix (2x8):")
    print(reshaped_2x8_matrix.eval())
    print("Reshaped matrix (4x4):")
    print(reshaped_4x4_matrix.eval())

Original matrix (8x2):
[[ 1  2]
 [ 3  4]
 [ 5  6]
 [ 7  8]
 [ 9 10]
 [11 12]
 [13 14]
 [15 16]]
Reshaped matrix (2x8):
[[ 1  2  3  4  5  6  7  8]
 [ 9 10 11 12 13 14 15 16]]
Reshaped matrix (4x4):
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]


In [19]:
#还可以使用 tf.reshape 更改张量的维数（\'阶\'）。 例如，您可以将 8x2 张量变形为三维 2x2x4 张量或一维 16 元素张量
#从二维变成三维或者一维
with tf.Graph().as_default():
  # Create an 8x2 matrix (2-D tensor).
  matrix = tf.constant([[1,2], [3,4], [5,6], [7,8],
                        [9,10], [11,12], [13, 14], [15,16]], dtype=tf.int32)

  # Reshape the 8x2 matrix into a 3-D 2x2x4 tensor.
  reshaped_2x2x4_tensor = tf.reshape(matrix, [2,2,4])
  
  # Reshape the 8x2 matrix into a 1-D 16-element tensor.
  one_dimensional_vector = tf.reshape(matrix, [16])

  with tf.Session() as sess:
    print("Original matrix (8x2):")
    print(matrix.eval())
    print("Reshaped 3-D tensor (2x2x4):")
    print(reshaped_2x2x4_tensor.eval())
    print("1-D vector:")
    print(one_dimensional_vector.eval())

Original matrix (8x2):
[[ 1  2]
 [ 3  4]
 [ 5  6]
 [ 7  8]
 [ 9 10]
 [11 12]
 [13 14]
 [15 16]]
Reshaped 3-D tensor (2x2x4):
[[[ 1  2  3  4]
  [ 5  6  7  8]]

 [[ 9 10 11 12]
  [13 14 15 16]]]
1-D vector:
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]


In [24]:
#改变两个张量的形状，使其能够相乘。
with tf.Graph().as_default():
    a = tf.constant([5, 3, 2, 7, 1, 4], dtype=tf.int32)
    b = tf.constant([4, 6, 3], dtype=tf.int32)
    #把上面两个不同型的函数整合成下面可以相乘的函数，注意，都要搞成二维的，也就是b也要reshape
    a_2x3 = tf.reshape(a,[2, 3])
    b_3x1 = tf.reshape(b, [3,1])
    
    result = tf.matmul(a_2x3 ,b_3x1)
    with tf.Session() as sess:
        print(result.eval())

[[44]
 [46]]


### 上面讲的都 是常量，我们下面看一下变量

In [27]:
g = tf.Graph()
with g.as_default():
  # Create a variable with the initial value 3.
  v = tf.Variable([3])

  # Create a variable of shape [1], with a random initial value,
  # sampled from a normal distribution with mean 1 and standard deviation 0.35.
  w = tf.Variable(tf.random_normal([1], mean=1.0, stddev=0.35))

#变量不会自动赋值，所以下面进行打印时会报错
with g.as_default():
  with tf.Session() as sess:
    try:
      v.eval()
    except tf.errors.FailedPreconditionError as e:
      print("Caught expected error: ", e)

Caught expected error:  Attempting to use uninitialized value Variable
	 [[Node: _retval_Variable_0_0 = _Retval[T=DT_INT32, index=0, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable)]]


 要初始化变量，最简单的方式是调用 `global_variables_initializer`。请注意 `Session.run()` 的用法（与 `eval()` 的用法大致相同）。

In [30]:
with g.as_default():
  with tf.Session() as sess:
    #初始化
    initialization = tf.global_variables_initializer()
    sess.run(initialization)
    # Now, variables can be accessed normally, and have values assigned to them.
    print(v.eval())
    print(w.eval())

[3]
[0.7344521]


In [31]:
with g.as_default():
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # This should print the variable's initial value.
    print(v.eval())
    #assign可以改变变量的值
    assignment = tf.assign(v, [7])
    # The variable has not been changed yet!
    print(v.eval())

    # Execute the assignment op.
    sess.run(assignment)
    # Now the variable is updated.
    print(v.eval())

[3]
[3]
[7]


 ### 练习 2：模拟投掷两个骰子 10 次。

创建一个骰子模拟，在模拟中生成一个 `10x3` 二维张量，其中：

  * 列 `1` 和 `2` 均存储一个骰子的一次投掷值。
  * 列 `3` 存储同一行中列 `1` 和 `2` 的值的总和。

例如，第一行中可能会包含以下值：

  * 列 `1` 存储 `4`
  * 列 `2` 存储 `3`
  * 列 `3` 存储 `7`

In [39]:
g = tf.Graph()
with g.as_default():
    
    with tf.Session() as sess:
        #tf.random_uniform(shape,minval=0,maxval=None,dtype=tf.float32)  产生一个随机张量，前面的shape可以规定生成的形状
        slice1 = tf.random_uniform([10],minval=1,maxval=7,dtype=tf.int32)
        slice2 = tf.random_uniform([10],minval=1,maxval=7,dtype=tf.int32) 
        slice3 = tf.add(slice1,slice2)
       
        
        a = tf.Variable([slice1,slice2,slice3])
        sess.run(tf.global_variables_initializer())
        print(a.eval())
    

[[ 3  6  2  6  1  1  1  3  5  4]
 [ 1  1  6  2  5  6  2  4  6  5]
 [ 4  7  8  8  6  7  3  7 11  9]]


In [44]:
g = tf.Graph()
with g.as_default():
    
    with tf.Session() as sess:
        #tf.random_uniform(shape,minval=0,maxval=None,dtype=tf.float32)  产生一个随机张量，前面的shape可以规定生成的形状
        slice1 = tf.random_uniform([5,5],minval=1,maxval=7,dtype=tf.int32)
        
        
        a = tf.Variable(slice1)
        sess.run(tf.global_variables_initializer())
        print(a.eval())

[[1 2 5 2 6]
 [1 3 4 5 1]
 [2 1 5 4 1]
 [2 5 6 1 1]
 [5 4 2 6 5]]
